# Résolution de Sudoku par Algorithme Génétique

## Introduction Théorique

Les algorithmes génétiques (GA) sont des techniques de recherche heuristiques inspirées par le processus de sélection naturelle. Ils sont couramment utilisés pour résoudre des problèmes d'optimisation et de recherche. Un algorithme génétique utilise des opérations telles que la mutation, le croisement et la sélection pour évoluer vers une solution optimale.

Dans le contexte du Sudoku, un algorithme génétique peut être utilisé pour trouver une solution en représentant une grille de Sudoku comme un chromosome, où chaque gène représente une cellule de la grille. Les opérations génétiques sont appliquées pour optimiser la grille en respectant les contraintes du Sudoku.

## Installation de GeneticSharp

Nous devons commencer par installer le package GeneticSharp.
Afin de concevoir un solver de Sudoku en algorithme génétique, il nous faudra concevoir un chromosome de Sudoku représentant l'individu d'une population de solution potentielles, et une fonction d'évaluation (fitness) évaluant la qualité d'un individu/chromosme.
GeneticSharp fournira tout le reste des composants nécessaires à la mise en oeuvre de l'algorithme.


In [1]:
#r "nuget: GeneticSharp"

Installed Packages GeneticSharp, 3.1.4

## Importation des Classes de Base

Nous allons importer les classes de base définies dans le notebook précédent, fournissant notamment la représentation, le chargement et l'affichage de Sudokus, et l'infrastructure de résolution.


In [2]:
#!import Sudoku-0-Environment.ipynb

# Notebook 0: Classes de Base pour la Résolution de Sudoku

Ce notebook contient les classes de base nécessaires pour la manipulation et la résolution des grilles de Sudoku. Il sera importé dans les autres notebooks pour réutiliser ces classes.

## Importation des Bibliothèques Nécessaires

Nous commençons par importer les bibliothèques nécessaires.


## Définition de la classe SudokuGrid

Nous définissons ici la classe SudokuGrid qui représente une grille de Sudoku et fournit des méthodes pour manipuler et afficher les grilles.


## Définition de l'interface ISudokuSolver

Nous définissons ici l'interface ISudokuSolver qui sera implémentée par les différentes stratégies de résolution de Sudoku.


## Définition de la classe SudokuHelper

Nous ajoutons ici la classe SudokuHelper qui contient des méthodes utilitaires pour charger et afficher des grilles de Sudoku.


## Affichage des Puzzles de chaque Difficulté

Nous allons charger et afficher un puzzle de chaque niveau de difficulté : Facile, Moyen et Difficile.


Puzzle Facile:
-------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Puzzle Moyen:
-------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Puzzle Difficile:
-------------------------------
| 4       |         | 8     5 | 
|    3    |         |         | 
|         | 7       |         | 
-------------------------------
|    2    |         |    6    | 
|         |    8    | 4       | 
|         |    1    |         | 
-------------------------------
|         | 6     3 |    7    | 
| 5       | 2       |         | 
| 1     4 |         |         | 
-------------------------------

## Implémentation du Solveur de Sudoku par Algorithme Génétique

Nous allons maintenant implémenter ce solveur en C#. Comme indiqué précédemment il nous faut code la notion de chromosome et de fonction d'évaluation.

Afin de nous donner la possibilité de tester plusieurs types de chromosome, on introduit une interface dédiée:

### Interface `ISudokuChromosome`


In [3]:
using GeneticSharp;

public interface ISudokuChromosome: IChromosome
{
	SudokuGrid TargetSudoku { get; set; }
    SudokuGrid GetSolution();
    
}

### Classe `SudokuFitness`

On peut maintenant proposer une fonction d'évaluation basée sur cette interface.
Cette classe évalue un chromosome en fonction du nombre d'erreurs dans la grille de Sudoku générée.


In [4]:
using GeneticSharp;

public class SudokuFitness : IFitness
{
    private SudokuGrid _targetSudokuGrid;

    public SudokuFitness(SudokuGrid targetSudokuGrid)
    {
        _targetSudokuGrid = targetSudokuGrid;
    }

    public double Evaluate(IChromosome chromosome)
    {
        var sudokuChromosome = (ISudokuChromosome)chromosome;
        var sudoku = sudokuChromosome.GetSolution();
        var nbErrors = sudoku.NbErrors(_targetSudokuGrid);
        return -nbErrors;
    }
}


### Classe `SudokuSolver`

Cette classe contient les méthodes pour résoudre le Sudoku en utilisant l'algorithme génétique.

Nous concevons dores et déjà une méthode de résolution qui supportera plusieurs types de chromosomes de Sudoku sur les bases de notre interface commune, et présentant des fonctionnalités avancées suivantes:

- Possibilité de fournir l'opérateur de croisement et de mutation (opérateurs uniformes par défaut)

- Utilisation du parallélisme. L'application des opérateurs génétiques à une population ainsi que l'évaluation de la population résultantes sont par nature parallélisables. GeneticSharp propose l'utilisation du parallélisme de tâche natif .Net pour l'utilisation optimale des coeurs de processeurs disponibles.

- Multi-Objectifs et ajustement de la taille de la population. L'algorithme se termine à l'issue d'une génération si le meilleur chromosome ne contient plus d'erreur, ou si sa fitness n'a pas évolué depuis un certain nombre de génération, signalant un possible effondrement de la diversité génétique sur un maximum local. Dans ce cas, l'algorithme est redémarré en doublant la taille de la population.   


In [5]:
using GeneticSharp;
using Microsoft.DotNet.Interactive;
using System;
using System.Diagnostics;

public class SudokuGeneticSolver: ISudokuSolver
{
    
    public SudokuGeneticSolver(ISudokuChromosome chromosome)
    {
        Chromosome = chromosome;
    }

    public ISudokuChromosome Chromosome { get; set; }

    // Opérateurs de mutation et de croisement
    public IMutation Mutation { get; set; } = new UniformMutation(true);
    public ICrossover Crossover { get; set; } = new UniformCrossover(0.5f);

    public SudokuGrid Solve(SudokuGrid s)
    {
        var maxDuration = 20;

        var fitness = new SudokuFitness(s);
        
        var populationSize = 400;

        //Opérateur de sélection (Elite rapide mais un peu sélectif)
        var selection = new EliteSelection ();

        // Critères de terminaison
        var fitnessThreshold = 0;
        int stableGenerationNb = 30;
        var termination = new OrTermination(new ITermination[]
        {
            new FitnessThresholdTermination(fitnessThreshold),
            new FitnessStagnationTermination(stableGenerationNb),
        });
        
        var nbErrors = 0;
        var stopWatch = Stopwatch.StartNew();
        var lastTime = stopWatch.Elapsed;
        DisplayedValue displayPlaceholder = null;

        SudokuGrid bestSudoku = null;
        do
        {
            Population population = new Population(populationSize, populationSize, Chromosome);

            GeneticAlgorithm ga = new GeneticAlgorithm(population, fitness, selection, Crossover, Mutation)
            {
                Termination = termination
            };
            // Opérateurs de parallélisation
            ga.OperatorsStrategy = new TplOperatorsStrategy();
            ga.TaskExecutor = new TplTaskExecutor();
            ga.GenerationRan += (sender, args) =>
            {
                var bestIndividual = (ISudokuChromosome)ga.Population.BestChromosome;
                bestSudoku = bestIndividual.GetSolution();
                nbErrors = bestSudoku.NbErrors(s);
                if (displayPlaceholder == null)
                {
                    displayPlaceholder = display($"Generation {ga.GenerationsNumber}, Population {ga.Population.CurrentGeneration.Chromosomes.Count}, NbErrors {nbErrors}, Elapsed {stopWatch.Elapsed - lastTime}");
                }
                else
                {
                displayPlaceholder.Update($"Generation {ga.GenerationsNumber}, Population {ga.Population.CurrentGeneration.Chromosomes.Count}, NbErrors {nbErrors}, Elapsed {stopWatch.Elapsed - lastTime}");
                }
            
            };
            lastTime = stopWatch.Elapsed;
            ga.Start();
            nbErrors = bestSudoku.NbErrors(s);
            populationSize = Math.Min(10000, populationSize *= 2);
        } while (nbErrors > 0 && stopWatch.Elapsed.TotalSeconds < maxDuration);

        return bestSudoku;

    }
    
}

### Premier chromosome simple

Pour ce premier essai, nous représentont chaque cellule par un gène à valeur de 1 à 9. L'initialisation tient compte du masque à résoudre, mais pas les opérateurs qui agissent au hasard.

In [6]:
using System;

public class SudokuCellsChromosome : ChromosomeBase, ISudokuChromosome
{
    public SudokuGrid TargetSudoku { get; set; }

    
    public SudokuCellsChromosome() : base(81) {}

    public override Gene GenerateGene(int geneIndex)
    {
        var row = geneIndex / 9;
        var col = geneIndex % 9;

        if (TargetSudoku.Cells[row, col] != 0)
        {
            return new Gene(TargetSudoku.Cells[row, col]);
        }

        var rnd = RandomizationProvider.Current;
        return new Gene(rnd.GetInt(1, 10));
    }

    public override IChromosome CreateNew()
    {
        var toReturn = new SudokuCellsChromosome(){TargetSudoku = TargetSudoku};
        toReturn.CreateGenes();
        return toReturn;
    }

    public SudokuGrid GetSolution()
    {
        var genes = GetGenes();
        var cells = new int[9, 9];
        for (int i = 0; i < 81; i++)
        {
            cells[i / 9, i % 9] = (int)genes[i].Value;
        }
        var sudoku = new SudokuGrid() { Cells = cells };
        return sudoku;
    }
}


## Test du Solveur

Nous allons maintenant tester notre solveur de Sudoku par algorithme génétique en utilisant des grilles de Sudoku de différentes difficultés : facile, moyen et difficile.


In [7]:
display("Test du solver genetique simple:");


display("Puzzle Sudoku Facile Initial:");

// Charger et tester un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();

//Création du chromosome:
var chromosome = new SudokuCellsChromosome(){TargetSudoku = easySudoku};

// Instanciation de Solver avec notre chromosmome SudokuCellsChromosome
var solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(easySudoku, solver);

Test du solver genetique simple:

Puzzle Sudoku Facile Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Generation 86, Population 1600, NbErrors 0, Elapsed 00:00:01.0479622

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 4769,0452 ms

## Conclusion Intermédiaire

Le solveur génétique a réussi à résoudre le Sudoku de difficulté facile en plusieurs redémarrages, prenant plusieurs secondes. Cela montre que notre approche fonctionne, mais elle n'est pas encore assez efficace pour résoudre des puzzles de difficultéplus difficile en un temps raisonnable.

### Introduction des Chromosomes Avancés

Pour améliorer l'efficacité de notre algorithme génétique, nous allons introduire des chromosomes avancés qui illustrent comment manipuler le problème pour réduire son espace de solution, et rendre possible les opérateurs génétiques disponiblespour une exploration plus efficace de l'espace des solutions.

Nous allons implémenter trois types de chromosomes avancés :
- **SudokuPermutationsChromosome** : Manipule les permutations des lignes.
- **SudokuPermutatedCellsChromosome** : Manipule les cellules permutées.
- **SudokuOrderedCellsChromosome** : Conçu pour utiliser les opérateurs de mutation et de croisement de type "Ordered".

Voyons comment implémenter ces chromosomes et les tester.


### Classe de Base : SudokuChromosomeBase

La classe SudokuChromosomeBase sert de fondation pour nos différents types de chromosomes de Sudoku avancés. Elle contient des fonctionnalités communes telles que la gestion de la grille de Sudoku cible, le traitement des permutations des chiffres de 1 à 9 pour les chromosomes qui en ont besoin, et la génération des gènes basés sur les contraintes du puzzle.

La première optimisation consiste à remarquer que les lignes de Sudokus sont à valeur dans l'ensemble des permutations des chiffres de 1 à 9, au nombre de 362880, qu'on peut filtrer pour chaque ligne pour ne garder que celles qui respectent le masque à résoudre en entrée.

Une bonne partie du code de cette classe de base consiste à calculer et mettre en cache l'ensemble des permutations et les ensembles filtrés pour en faire le nouveau matériel génétique manipulable. 

In [8]:
#nullable enable
using System;
using System.Collections.Generic;
using System.Linq;
using GeneticSharp;

public abstract class SudokuChromosomeBase : ChromosomeBase, ISudokuChromosome
{
    public abstract bool UsesPermutations();

    public SudokuGrid TargetSudoku {get;set;}
    
    private Dictionary<(int row, int col), List<int>>? _extendedMask;
    private IList<IList<IList<int>>>? _targetRowsPermutations; // Rendons cette propriété nullable
    private IList<IList<IList<int>>> _RowsPermutations = new List<IList<IList<int>>>();
    private IList<int> _permutationsGenes = new List<int>();
    private IList<IList<int>> _permutations = new List<IList<int>>();


    // public SudokuChromosomeBase(int length) : base(length){}

    protected SudokuChromosomeBase(SudokuGrid targetSudokuGrid, Dictionary<(int row, int col), List<int>>? extendedMask, int length) : base(length)
    {
        TargetSudoku = targetSudokuGrid;
        _extendedMask = extendedMask;
    }

    public IList<int> PermutationsGenes => _permutationsGenes;
    public IList<IList<int>> Permutations => _permutations;

    public bool CheckPermutation(IList<int> permutation, int row)
    {
        var toReturn = true;
        for (int i = 0; i < 9; i++)
        {
            if (TargetSudoku != null && TargetSudoku.Cells[row, i] != 0)
            {
                if (permutation[i] != TargetSudoku.Cells[row, i])
                {
                    toReturn = false;
                    break;
                }
            }
        }
        return toReturn;
    }

    protected override void CreateGenes()
    {
        var rnd = RandomizationProvider.Current;

        if (UsesPermutations())
        {
            for (int index = 0; index < 9; ++index)
            {
                IList<IList<int>> permutations = TargetRowsPermutations[index];
                IList<IList<int>> rowPermutations = _RowsPermutations[index];

                int[] permTargetIdxs = permutations.Where((perm) => CheckPermutation(perm, index)).Select((perm, idx) => idx).ToArray();
                int permIdx;
                if (permTargetIdxs.Length != 0)
                {
                    permIdx = rowPermutations.IndexOf(permutations[permTargetIdxs[rnd.GetInt(0, permTargetIdxs.Length)]]);
                }
                else
                {
                    permIdx = rnd.GetInt(0, rowPermutations.Count);
                }

                _permutationsGenes.Add(permIdx);
                _permutations.Add(rowPermutations[permIdx]);
            }
        }

        for (int index = 0; index < this.Length; ++index)
            this.ReplaceGene(index, this.GenerateGene(index));
    }


    public IList<IList<IList<int>>> GetRowsPermutations()
    {
        if (TargetSudoku == null)
        {
            return UnfilteredPermutations;
        }

        if (!_rowsPermutations.TryGetValue(TargetSudoku, out var toReturn))
        {
            lock (_rowsPermutations)
            {
                if (!_rowsPermutations.TryGetValue(TargetSudoku, out toReturn))
                {
                    toReturn = GetRowsPermutationsUncached();
                    _rowsPermutations[TargetSudoku] = toReturn;
                }
            }
        }
        if (this._RowsPermutations.Count == 0)
            this._RowsPermutations = toReturn;
        return toReturn;
    }

    private IList<IList<IList<int>>> GetRowsPermutationsUncached()
    {
        var toReturn = new List<IList<IList<int>>>(9);
        for (int i = 0; i < 9; i++)
        {
            var tempList = new List<IList<int>>();
            foreach (var perm in AllPermutations)
            {
                if (Range9.All(j => ExtendedMask[(i, j)].Contains(perm[j])))
                {
                    tempList.Add(perm);
                }
            }
            toReturn.Add(tempList);
        }

        return toReturn;
    }

    public static IList<IList<IList<int>>> UnfilteredPermutations
    {
        get
        {
            if (!_unfilteredPermutations.Any())
            {
                lock (_unfilteredPermutations)
                {
                    if (!_unfilteredPermutations.Any())
                    {
                        _unfilteredPermutations = Range9.Select(i => AllPermutations).ToList();
                    }
                }
            }
            return _unfilteredPermutations;
        }
    }

    public static IList<IList<int>> AllPermutations
    {
        get
        {
            if (!_allPermutations.Any())
            {
                lock (_allPermutations)
                {
                    if (!_allPermutations.Any())
                    {
                        _allPermutations = GetPermutations(Enumerable.Range(1, 9), 9);
                    }
                }
            }
            return _allPermutations;
        }
    }

    public IList<IList<IList<int>>> TargetRowsPermutations
    {
        get
        {
            if (_targetRowsPermutations == null)
            {
                _targetRowsPermutations = GetRowsPermutations();
            }
            return _targetRowsPermutations;
        }
    }

    private static readonly IDictionary<SudokuGrid, IList<IList<IList<int>>>> _rowsPermutations = new Dictionary<SudokuGrid, IList<IList<IList<int>>>>();
    private static readonly List<int> Range9 = Enumerable.Range(0, 9).ToList();
    private static IList<IList<int>> _allPermutations = new List<IList<int>>();
    private static IList<IList<IList<int>>> _unfilteredPermutations = new List<IList<IList<int>>>();

    static IList<IList<T>> GetPermutations<T>(IEnumerable<T> list, int length)
    {
        if (length == 1) return list.Select(t => (IList<T>)(new T[] { t }.ToList())).ToList();

        var enumeratedList = list.ToList();
        return (IList<IList<T>>)GetPermutations(enumeratedList, length - 1)
            .SelectMany(t => enumeratedList.Where(e => !t.Contains(e)),
                (t1, t2) => (IList<T>)t1.Concat(new T[] { t2 }).ToList()).ToList();
    }

    public Dictionary<(int row, int col), List<int>> ExtendedMask
    {
        get
        {
            if (_extendedMask == null)
                BuildExtendedMask();

            return _extendedMask!;
        }
    }

    private void BuildExtendedMask()
    {
        var indices = Enumerable.Range(1, 9).ToList();
        var extendedMask = new Dictionary<(int row, int col), List<int>>(81);
        if (TargetSudoku != null)
        {
            var forbiddenMask = new Dictionary<(int row, int col), List<int>>();
            for (var row = 0; row < 9; row++)
            {
                for (var col = 0; col < 9; col++)
                {
                    int targetCell = TargetSudoku.Cells[row, col];
                    if (targetCell != 0)
                    {
                        var cellNeighbours = SudokuGrid.CellNeighbours[row][col];
                        foreach (var cellNeighbour in cellNeighbours)
                        {
                            if (!forbiddenMask.TryGetValue(cellNeighbour, out var targetList))
                            {
                                targetList = new List<int>();
                                forbiddenMask[cellNeighbour] = targetList;
                            }
                            if (!targetList.Contains(targetCell))
                            {
                                targetList.Add(targetCell);
                            }
                        }
                    }
                }
            }

            for (var row = 0; row < 9; row++)
            {
                for (var col = 0; col < 9; col++)
                {
                    var cellIndex = (row, col);
                    extendedMask[cellIndex] = indices.Where(i => !forbiddenMask[cellIndex].Contains(i)).ToList();
                }
            }
        }
        else
        {
            for (var row = 0; row < 9; row++)
            {
                for (var col = 0; col < 9; col++)
                {
                    var cellIndex = (row, col);
                    extendedMask[cellIndex] = indices;
                }
            }
        }
        _extendedMask = extendedMask;
    }

    public abstract SudokuGrid GetSolution();
}


### Chromosome : SudokuPermutationsChromosome

Le SudokuPermutationsChromosome est un chromosome simple de 9 gènes qui manipule les permutations de lignes du Sudoku. Chaque gène représente une permutation d'une ligne entière, permettant ainsi une exploration plus efficace des solutions potentielles.

In [9]:
public class SudokuPermutationsChromosome : SudokuChromosomeBase
{
    public SudokuPermutationsChromosome() : base(null, null, 9) { }

    public SudokuPermutationsChromosome(Dictionary<(int row, int col), List<int>> extendedMask)
        : base(null, extendedMask, 9)
    {
    }

    public override Gene GenerateGene(int geneIndex)
    {
        Gene toReturn = new Gene(PermutationsGenes[geneIndex]);
        return toReturn;
    }

    public override IChromosome CreateNew()
    {
        var toReturn = new SudokuPermutationsChromosome(ExtendedMask){TargetSudoku = TargetSudoku};
        toReturn.CreateGenes();
        return toReturn;
    }

    public override bool UsesPermutations()
    {
        return true;
    }

    public override SudokuGrid GetSolution()
    {
        var listInt = new List<int[]>(81);
        for (int i = 0; i < 9; i++)
        {
            var perm = GetPermutation(i).ToArray();
            listInt.Add(perm);
        }
        var sudoku = new SudokuGrid() { Cells = SudokuGrid.To2D(listInt.ToArray()) };
        return sudoku;
    }

    protected virtual List<int> GetPermutation(int rowIndex)
    {
        int permIDx = GetPermutationIndex(rowIndex);
        return GetPermutation(rowIndex, permIDx);
    }

    protected virtual List<int> GetPermutation(int rowIndex, int permIDx)
    {
        var perm = GetRowsPermutations()[rowIndex][permIDx % GetRowsPermutations()[rowIndex].Count].ToList();
        return perm;
    }

    protected virtual int GetPermutationIndex(int rowIndex)
    {
        return (int)GetGene(rowIndex).Value;
    }
}


### Test du SudokuPermutationsChromosome

Nous allons maintenant tester le SudokuPermutationsChromosome en utilisant notre algorithme génétique. Nous allons voir comment il se comporte sur un Sudoku de difficulté facile, moyenne et difficile.

In [10]:
display("Test du solver genetique de permutations de lignes:");


display("Puzzle Sudoku Facile Initial:");

// Charger et tester un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();

//Création du chromosome:
var chromosome = new SudokuPermutationsChromosome(){TargetSudoku = easySudoku};

// Instanciation de Solver avec notre chromosmome SudokuCellsChromosome
var solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(easySudoku, solver);

display("Puzzle Sudoku Medium Initial:");

// Charger et tester un puzzle moyen
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First();

chromosome = new SudokuPermutationsChromosome(){TargetSudoku = mediumSudoku};
solver = new SudokuGeneticSolver(chromosome);

SudokuHelper.SolveSudoku(mediumSudoku, solver);

Test du solver genetique de permutations de lignes:

Puzzle Sudoku Facile Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Generation 1, Population 400, NbErrors 0, Elapsed 00:00:01.4848566

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 1485,1341 ms

Puzzle Sudoku Medium Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Generation 61, Population 3200, NbErrors 4, Elapsed 00:00:09.5002400

Sudoku renvoyé:
-------------------------------
| 8  5  1 | 6  9  2 | 4  3  7 | 
| 7  2  3 | 5  1  4 | 8  6  9 | 
| 9  6  4 | 8  7  3 | 1  2  5 | 
-------------------------------
| 9  8  6 | 1  4  7 | 5  3  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 1  4  2 | 3  5  9 | 7  8  6 | 
-------------------------------
| 4  3  2 | 9  8  1 | 6  7  5 | 
| 6  1  7 | 4  2  5 | 3  9  8 | 
| 5  9  8 | 7  3  6 | 2  4  1 | 
-------------------------------
Nombre d'erreurs réstantes: 4
Temps de résolution: 20495,5069 ms

On constate un progrès mais l'agorithme est encore insuffisant pour les Sudokus plus difficiles.

Le chromosome suivant introduit une suite ordonnée de gènes, afin de pouvoir utiliser les opérateurs génétiques dédiés.

In [13]:
public class SudokuOrderedCellsChromosome : SudokuChromosomeBase
    {
	    public IList<(int row, int col)> GeneToCellLookup
	    {
		    get
		    {
			    if (_geneToCellLookup == null)
			    {
					if (TargetSudoku == null)
					{
						_geneToCellLookup = Enumerable.Range(0, 81).Select(x => (x / 9, x % 9)).ToList();
					}
					else
					{
						_geneToCellLookup = new List<(int row, int col)>(Length);
						for (int i = 0; i < 9; i++)
						{
							for (int j = 0; j < 9; j++)
							{
								if (TargetSudoku.Cells[i,j] == 0)
								{
									GeneToCellLookup.Add((i, j));
								}
								else
								{
									// Add 1 to the lookup table for each number that is already in the board
									this.baseLookupTable[TargetSudoku.Cells[i,j] - 1]++;
								}
							}
						}
					}
					this.cloneLookupTable = new List<int>(baseLookupTable);
				}
			    return _geneToCellLookup;
		    }
	    }

	    private List<int> baseLookupTable = new List<int>(9) { 0, 0, 0, 0, 0, 0, 0, 0, 0 };
	    private List<int> cloneLookupTable;
	    private IList<(int row, int col)> _geneToCellLookup;

	    public SudokuOrderedCellsChromosome()
            : this(null)
        {
        }

        public SudokuOrderedCellsChromosome(SudokuGrid targetSudokuBoard)
            : this(targetSudokuBoard, null)
        {
        }

        public SudokuOrderedCellsChromosome(
	        SudokuGrid targetSudokuBoard,
            Dictionary<(int x, int y), List<int>> extendedMask)
            : base(targetSudokuBoard, extendedMask, targetSudokuBoard?.NbEmptyCells()?? 81)
        {
        }

        public SudokuOrderedCellsChromosome(
	        SudokuGrid targetSudokuBoard,
	        Dictionary<(int x, int y), List<int>> extendedMask,
	        IList<(int row, int col)> objGeneToCellLookup,
	        List<int> objBaseLookupTable)
	        : base(targetSudokuBoard, extendedMask, targetSudokuBoard?.NbEmptyCells() ?? 81)
        {
            this._geneToCellLookup = objGeneToCellLookup;
            this.baseLookupTable = objBaseLookupTable;
            this.cloneLookupTable = new List<int>(objBaseLookupTable);
        }

        public static Random Random = new Random();

        public override Gene GenerateGene(int geneIndex)
        {
	        var targetCell = GeneToCellLookup[geneIndex];

	        var availableFromLookup = this.cloneLookupTable.Select(( value, ind) => (value, ind))
		        .Where((tuple => tuple.value < 9)).Select(tuple => tuple.ind+1).ToArray();
            var availableFromCoherence = this.ExtendedMask[(targetCell.row, targetCell.col)];
            var crossedAvail = availableFromCoherence.Where(i => availableFromLookup.Contains(i)).ToArray();
            int figureValue;
			if (crossedAvail.Length==0)
            {
	            // If no value is available from the mask and the lookuptable, we take the minimum value (index + 1) from the lookup table
				figureValue= this.cloneLookupTable.Select((value, ind) => (value, ind)).MinBy(tuple => tuple.value).ind + 1;
            }
            else
            {
	            // If there are values available from the mask and the lookuptable, we take a random value from the crossed list
				var figureIndex = Random.Next(crossedAvail.Count());
				figureValue = crossedAvail[figureIndex];
			}
           
            var geneValue = cloneLookupTable[figureValue-1]*9 + figureValue-1;
            cloneLookupTable[figureValue-1] += 1;

            Gene gene = new Gene(geneValue);
            return gene;
        }

        public override IChromosome CreateNew()
        {
	        var toReturn = new SudokuOrderedCellsChromosome(this.TargetSudoku, this.ExtendedMask,
		        GeneToCellLookup, baseLookupTable);
			toReturn.CreateGenes();
			return toReturn;
        }

        public override bool UsesPermutations()
        {
	        return false;
        }

        public override SudokuGrid GetSolution()
        {
            var computedSudoku = (SudokuGrid) TargetSudoku.Clone();
            var genes = GetGenes();
            for (int geneIndex = 0; geneIndex < this.Length; geneIndex++)
            {
                var cellIndex = GeneToCellLookup[geneIndex];
				computedSudoku.Cells[cellIndex.row,cellIndex.col] = (int)genes[geneIndex].Value % 9 + 1 ;
			}
            return computedSudoku;
        }
    }

### Test du OrderedCellsChromosome

Nous allons maintenant tester le SudokuPermutationsChromosome en utilisant notre algorithme génétique. Nous allons voir comment il se comporte sur un Sudoku de difficulté facile, moyenne et difficile.

In [14]:
display("Test du solver genetique de permutations de lignes:");


display("Puzzle Sudoku Facile Initial:");

// Charger et tester un puzzle facile
var easySudokus = SudokuHelper.GetSudokus(SudokuDifficulty.Easy);
var easySudoku = easySudokus.FirstOrDefault();

//Création du chromosome:
var chromosome = new SudokuOrderedCellsChromosome(){TargetSudoku = easySudoku};

var mutation = new TworsMutation();
var crossover = new AlternatingPositionCrossover  ();

var solver = new SudokuGeneticSolver(chromosome){Mutation = mutation, Crossover = crossover};

SudokuHelper.SolveSudoku(easySudoku, solver);

display("Puzzle Sudoku Medium Initial:");

// Charger et tester un puzzle moyen
var mediumSudoku = SudokuHelper.GetSudokus(SudokuDifficulty.Medium).First();

chromosome = new SudokuOrderedCellsChromosome(){TargetSudoku = mediumSudoku};
solver =  new SudokuGeneticSolver(chromosome){Mutation = mutation, Crossover = crossover};

SudokuHelper.SolveSudoku(mediumSudoku, solver);

Test du solver genetique de permutations de lignes:

Puzzle Sudoku Facile Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 9     2 |       5 | 4     3 | 
| 1       |    6  3 |    2  5 | 
| 5     8 | 4     7 |    6    | 
-------------------------------
|    2  6 | 3     9 |       1 | 
|    5  7 |    1    | 2  9    | 
|    9    | 6  7    | 5  3    | 
-------------------------------
| 2  4    | 5  3    | 6       | 
| 7     5 | 2       | 3     4 | 
|    8    |    4  1 | 9  5    | 
-------------------------------

Generation 81, Population 400, NbErrors 0, Elapsed 00:00:00.9636294

Sudoku renvoyé:
-------------------------------
| 9  6  2 | 1  8  5 | 4  7  3 | 
| 1  7  4 | 9  6  3 | 8  2  5 | 
| 5  3  8 | 4  2  7 | 1  6  9 | 
-------------------------------
| 8  2  6 | 3  5  9 | 7  4  1 | 
| 3  5  7 | 8  1  4 | 2  9  6 | 
| 4  9  1 | 6  7  2 | 5  3  8 | 
-------------------------------
| 2  4  9 | 5  3  8 | 6  1  7 | 
| 7  1  5 | 2  9  6 | 3  8  4 | 
| 6  8  3 | 7  4  1 | 9  5  2 | 
-------------------------------
Nombre d'erreurs réstantes: 0
Temps de résolution: 963,83 ms

Puzzle Sudoku Medium Initial:

Résolution par le solver SudokuGeneticSolver du Sudoku:
 -------------------------------
| 8  5    |       2 | 4       | 
| 7  2    |         |       9 | 
|       4 |         |         | 
-------------------------------
|         | 1     7 |       2 | 
| 3     5 |         | 9       | 
|    4    |         |         | 
-------------------------------
|         |    8    |    7    | 
|    1  7 |         |         | 
|         |    3  6 |    4    | 
-------------------------------

Generation 284, Population 3200, NbErrors 9, Elapsed 00:00:19.2799666

Sudoku renvoyé:
-------------------------------
| 8  5  6 | 7  9  2 | 4  3  1 | 
| 7  2  1 | 6  4  3 | 8  5  9 | 
| 9  3  4 | 8  5  1 | 7  2  6 | 
-------------------------------
| 6  9  8 | 1  4  7 | 5  3  2 | 
| 3  7  5 | 2  6  8 | 9  1  4 | 
| 2  4  1 | 5  7  9 | 6  8  3 | 
-------------------------------
| 4  6  3 | 9  8  1 | 2  7  5 | 
| 9  1  7 | 4  2  5 | 3  6  8 | 
| 5  8  2 | 7  3  6 | 1  4  9 | 
-------------------------------
Nombre d'erreurs réstantes: 9
Temps de résolution: 32548,2492 ms